In [180]:
%run paths.ipynb
%run preprocess.ipynb

In [181]:
p = parseNeuralSum()

In [182]:
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_TRAIN_FOLDER, target_document_folder=CONFIG.CNN_TRAIN_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_TRAIN_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_TRAIN_SUMMARY_FOLDER)
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_TEST_FOLDER, target_document_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_TEST_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_TEST_SUMMARY_FOLDER)
p.parse_folder_1(input_folder=CONFIG.RAW_CNN_VALIDATION_FOLDER, target_document_folder=CONFIG.CNN_VALIDATION_DOCUMENTS_FOLDER, 
                 target_output_folder=CONFIG.CNN_VALIDATION_OUTPUT_FOLDER, target_summary_folder=CONFIG.CNN_VALIDATION_SUMMARY_FOLDER)

In [124]:
g = NeuralSumGlove(glove_filename=CONFIG.GLOVE_FILENAME)

In [125]:
g.convert_to_indices_(document_folder=CONFIG.CNN_TRAIN_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_TRAIN_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_TRAIN_DUMP)
g.convert_to_indices_(document_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_TEST_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_TEST_DUMP)
g.convert_to_indices_(document_folder=CONFIG.CNN_VALIDATION_DOCUMENTS_FOLDER, output_folder=CONFIG.CNN_VALIDATION_OUTPUT_FOLDER, 
                     dump_file=CONFIG.CNN_VALIDATION_DUMP)

In [126]:
g.dump_all(CONFIG.GLOVE_CACHE_FILENAME)

In [127]:
d = NeuralSumDataHandler()

In [128]:
d.load_dump(CONFIG.CNN_TRAIN_DUMP)
d.pad(output=True)
d.padded_dump(CONFIG.CNN_PADDED_TRAIN_DUMP)
d.load_dump(CONFIG.CNN_TEST_DUMP)
d.pad(output=True)
d.padded_dump(CONFIG.CNN_PADDED_TEST_DUMP)

**RUN FROM HERE FOR TRAINING**

In [129]:
%run paths.ipynb
%run preprocess.ipynb
%run model.ipynb
%run utils.ipynb

In [130]:
d = NeuralSumDataHandler()
d.load_padded_dump(CONFIG.CNN_PADDED_TRAIN_DUMP)
d.make_batches(20)
g = NeuralSumGlove(dump_filename=CONFIG.GLOVE_CACHE_FILENAME)
torch.cuda.set_device(0)

In [6]:
m = EncoderDecoder(g.vectors, word_emb_size=g.dim, sen_emb_size=150, doc_emb_size=200, sen_len=50, batch_size=20, output_dim=1)
m.cuda()

EncoderDecoder(
  (sentenceEncoder): LSTMSentenceEncoderParallel(
    (embeddings): Embedding(400001, 100, padding_idx=400000)
    (sentenceEncoder): LSTM(100, 150, batch_first=True)
  )
  (documentEncoder): LSTM(150, 200, batch_first=True)
  (documentDecoder): LSTM(150, 200, batch_first=True)
  (classifier): Linear(in_features=200, out_features=1, bias=True)
)

In [ ]:
train_classifier(model=m, lines=d.lines, output=d.output, batches=d.batches, iterations=10)
torch.save(m.state_dict(), 'parameters')

**RUN FROM HERE FOR TESTING**

In [171]:
%run model.ipynb
%run utils.ipynb

In [140]:
m.load_state_dict(torch.load('parameters'))

In [141]:
d = NeuralSumDataHandler()
d.load_padded_dump(CONFIG.CNN_PADDED_TEST_DUMP)
d.make_batches(20)
g = NeuralSumGlove(dump_filename=CONFIG.GLOVE_CACHE_FILENAME)

In [142]:
m = EncoderDecoder(g.vectors, word_emb_size=g.dim, sen_emb_size=150, doc_emb_size=200, sen_len=50, batch_size=20, output_dim=1)
m.cuda()

EncoderDecoder(
  (sentenceEncoder): LSTMSentenceEncoderParallel(
    (embeddings): Embedding(400001, 100, padding_idx=400000)
    (sentenceEncoder): LSTM(100, 150, batch_first=True)
  )
  (documentEncoder): LSTM(150, 200, batch_first=True)
  (documentDecoder): LSTM(150, 200, batch_first=True)
  (classifier): Linear(in_features=200, out_features=1, bias=True)
)

In [172]:
get_summaries(m, d.lines, d.batches, doc_folder=CONFIG.CNN_TEST_DOCUMENTS_FOLDER, write_folder=CONFIG.PREDICTED_SUMMARY_FOLDER)

In [178]:
get_summary(100, CONFIG.PREDICTED_SUMMARY_FOLDER)

tensor([ 9,  0, 10], device='cuda:0')
they were led by a man wearing a " @entity50 " t - shirt who shouted , " this is what democracy looks like ! " @entity11 's shooting stirred memories of the @entity54 case in @entity55 , @entity56 , where an unarmed black teenager was killed by a white police officer
the officer charged with murder in the shooting death of an unarmed black man in @entity5 has been fired as anger continues to build around his case
a grand jury declined to indict the officer in that case



In [183]:
get_summary(100, CONFIG.CNN_TEST_SUMMARY_FOLDER)

witness who took the video says " mr. @entity11 did n't deserve this "
@entity22 police officer @entity9 is fired
the city orders an additional 150 body cameras

